In [2]:
#read in all .json files
import os
DATA_LOC= f"..{os.sep}Data{os.sep}Pending"
file_ext=".json"
files=[]
for file in os.listdir(DATA_LOC):
    if file.endswith(file_ext):
        files.append(os.path.join(DATA_LOC, file))



In [3]:
#process a file
import json
facts=[]
misinformation=[]
temporal=[]
ignore=[]
opinion=[]
none=[]
errors=[]

for file in files:
    with open(file,encoding='utf8') as f:
        data = json.load(f)

        sentences=data['sentences']

        for sentence in sentences:
            annotated=False
            if sentence['fact'] and sentence['misinformation']:
                errors.append(sentence)
                annotated=True
            elif sentence['misinformation'] and sentence['opinion']:
                errors.append(sentence)
                annotated=True
            elif sentence['fact']:
                facts.append(sentence)
                annotated=True
            elif sentence['misinformation']:
                misinformation.append(sentence)
                annotated=True
            elif sentence['opinion']:
                opinion.append(sentence)
                annotated=True
            if sentence['temporal']:
                temporal.append(sentence)
                annotated=True
            if sentence['ignore']:
                ignore.append(sentence)
                annotated=True
            if not annotated:
                none.append(sentence)
           
    
print(f'# of Facts: {len(facts)}')
print(f'# of Misinformation: {len(misinformation)}')
print(f'# of Opinion: {len(opinion)}')
print(f'# of temporal: {len(temporal)}')
print(f'# of ignore: {len(ignore)}')
print(f'# of not annotated: {len(none)}')
print(f'# of errors: {len(errors)}')
    

# of Facts: 3221
# of Misinformation: 141
# of Opinion: 4105
# of temporal: 308
# of ignore: 3439
# of not annotated: 23179
# of errors: 64


In [4]:
import nltk
import statistics
def get_text(annotations):
    tokenized_text=[]  
    for d in annotations:  
        tokenized_text.append(nltk.pos_tag(nltk.word_tokenize(d['sentence'])))
    return tokenized_text   
def pos_analysis(data):    
    text=get_text(data)

    word_freq={}
    pos_freq={}
    lengths=[]
    total=0
    for sentence in text:
        lengths.append(len(sentence))
        for word in sentence:
            total+=1
            if word[0] not in word_freq:
                word_freq[word[0].lower()] = 0 
            word_freq[word[0].lower()] += 1
            if word[1] not in pos_freq:
                pos_freq[word[1]] = 0 
            pos_freq[word[1]] += 1

    sorted_word_freq = sorted(word_freq.items(), key=lambda kv: kv[1],reverse=True)
    ''' 
    for x in sorted_word_freq[:10]:
        print(f'{x}')
    '''
    sorted_pos_freq = sorted(pos_freq.items(), key=lambda kv: kv[1],reverse=True)
    
    posnumbers={}
    for x in sorted_pos_freq[:20]:
       # print(f'{x[0]} : {x[1]/total*100}')
        posnumbers[x[0]]=x[1]/total*100
    return posnumbers

    
   # return statistics.mean(lengths)
results={}     
results['temporal']=pos_analysis(temporal)
results['opinion']=pos_analysis(opinion)
results['misinformation']=pos_analysis(misinformation)
results['facts']=pos_analysis(facts)
results['none']=pos_analysis(none)



In [6]:
from sklearn import svm
import random,math
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()
def tokenize(text):
    text=text.lower()
    tokens = nltk.word_tokenize(text)
   # stems = []
   # for item in tokens:
   #     stems.append(PorterStemmer().stem(item))
    #return stems
    return tokens
def tokenize_stem(text):
    text=text.lower()
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

def tokenize_pos(text):
    text=text.lower()
    tokens = nltk.word_tokenize(text)
    pos=nltk.pos_tag(tokens)
    tags=[]
    for p in pos:
        tags.append(p[1])
    return tags

def tokenize_pos_combo(text):
    text=text.lower()
    tokens = nltk.word_tokenize(text)
    pos=nltk.pos_tag(tokens)
    tags=[]
    for p in pos:
        tags.append(p[0])
        tags.append(p[1])
    return tags


data =[]
y=[]

data.extend([d['sentence'] for d in facts])
y.extend(['fact']*len(facts))
#y.extend(['statement']*len(facts))

data.extend([d['sentence'] for d in opinion])
y.extend(['opinion']*len(opinion))
#y.extend(['']*len(opinion))

data.extend([d['sentence'] for d in misinformation])
y.extend(['misinformation']*len(misinformation))
#y.extend(['statement']*len(misinformation))


data.extend([d['sentence'] for d in ignore])
y.extend(['ignore']*len(ignore))

data.extend([d['sentence'] for d in none])
y.extend(['none']*len(none))

labels = ['fact','opinion','misinformation','ignore','none']
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=1, max_df=0.9,ngram_range=(1, 3),tokenizer=tokenize,)

X = tfidfconverter.fit_transform(data).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=0)
classifier.fit(X_train, y_train) 





RandomForestClassifier(random_state=0)